In [3]:
from bs4 import BeautifulSoup, Tag, NavigableString
import json, os
def open_json(file):
    data = {}
    if os.path.exists(file):
        with open(file, 'r', encoding="UTF-8") as fp:
            data = json.load(fp)
            fp.close()
    return data
def save_json(file, data):
    with open(file, 'w', encoding="UTF-8") as fp:
        json.dump(data, fp, indent=4)
        fp.close()

raw_catalog_data = open_json(f'../data/lad_catalog/raw/catalog_item_tables.json')

In [4]:
def scrape_metadatatables(tables: list[BeautifulSoup|Tag|NavigableString], source: str) -> dict:
    result = {
        'unknown': [],
        'source': source
    }
    for table in tables:
        for row in table.find_all("tr"):
            try:
                th = row.find("th")
                td = row.find("td")
                if td is None:
                    continue
                
                if th is None:
                    result['unknown'].append(td)
                name = th.text.strip()
                multi_column_items = td.find_all(attrs={'class': 'ce-multicolumn-field__item'})
                if len(multi_column_items) == 0:
                    result[name] = td.text.strip()
                else:
                    for item in multi_column_items:
                        label = item.find(attrs={'class': 'ionic-icon__label'})
                        value = item.find(attrs={'class': 'ionic-icon'})
                        if 'checkbox' in value.text.strip().lower():
                            if name not in result:
                                result[name] = []
                            result[name].append(label.text.strip())
                        elif 'on' in value.text.strip().lower():
                             result[name] = label.text.strip()
                    if name not in result:
                        result[name] = multi_column_items.text.strip()   
            except:
                print(f"{source} -> {row}")
        
    return result

In [6]:
catalog_data = []
for source, tables in raw_catalog_data.items():
    bs_tables = list(map(lambda table: BeautifulSoup(table, features="html.parser"), tables))
    catalog_data.append(scrape_metadatatables(bs_tables, source))

catalog_data

https://eps.lad.gov.lv/show_etk_public?action=index&controller=catalog_entries_public&id=10771&page=18 -> <tr>
<th class="ce-fixed-column">
   Griezējinstruments
  </th>
<td>
<div class="ce-multicolumn-field__column">
<div class="ce-multicolumn-field__item">
<div class="ionic-icon">
<svg class="ionicon" viewbox="0 0 512 512" xmlns="http://www.w3.org/2000/svg">
<title>
        Square
       </title>
<path d="M416 448H96a32.09 32.09 0 01-32-32V96a32.09 32.09 0 0132-32h320a32.09 32.09 0 0132 32v320a32.09 32.09 0 01-32 32z" fill="none" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="32">
</path>
</svg>
</div>
<div class="ionic-icon__label">
<label for="pas_vais">
       pasīvais
      </label>
</div>
</div>
<div class="ce-multicolumn-field__item">
<div class="ionic-icon">
<svg class="ionicon" viewbox="0 0 512 512" xmlns="http://www.w3.org/2000/svg">
<title>
        Square
       </title>
<path d="M416 448H96a32.09 32.09 0 01-32-32V96a32.09 32.09 0 0132-32

[{'unknown': [],
  'source': 'https://eps.lad.gov.lv/show_etk_public?action=index&controller=catalog_entries_public&id=2474&page=1',
  'Numurs': 'TR-01-0000086',
  'Kategorija': 'Traktori',
  'Tehnikas vienība': 'Lauksaimniecības traktors',
  'Marka': 'FENDT',
  'Modelis': '512 VARIO',
  'Aprīkojuma līmenis': 'Premium',
  'Aprīkojuma apraksts': '4.0 L dzinējs, Priekšējā tilta amortizācija, Kabīnes amortizācija, Priekšējā uzkare, 12 collu vadības terminālis, Bākuguns, Darba apgaismojuma komplekts, ISOBUS savienojums, Automātiskā stūrēšanas sistēma',
  'Cena bez PVN, EUR': '150500.00',
  'Dzinējs - jauda, Zs': '124',
  'Dzinējs - cilindru skaits': '4',
  'Riteņu formula': 'kāpurķēžu',
  'Transmisija - Pārnesumu skaits (uz priekšu)': '0',
  'Transmisija - Pārnesumu skaits (atpakaļ)': '0',
  'Transmisija - bezpakāpju': 'Jā',
  'Hidrosūkņa ražīgums, l/min': '158.00',
  'Uzkares celtspēja, kg': '8045.00'},
 {'unknown': [],
  'source': 'https://eps.lad.gov.lv/show_etk_public?action=index&cont

In [13]:
import re
def open_csv(file, sep=';'):
    fs = open(file, 'r', encoding='utf-8')
    data = list(map(lambda row: row.split(sep),fs.readlines()))
    fs.close()
    return data
replacements = {
    'ā': 'a',
    'ē': 'e',
    'č': 'c',
    'ī': 'i',
    'ļ': 'l',
    'š': 's',
    'ū': 'u',
    'ģ': 'g',
    'ķ': 'k',
    'ž': 'z',
    'ņ': 'n'
}

def normalize_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    for r in replacements:
        if r in text:
            text = text.replace(r, replacements[r])
    return text

In [20]:
data = open_csv('../static/remaining_value_combine.csv')
result = {}
header = data[0][1:]
for line in data[1:]:
    percentages = list(map(lambda v: float(v.strip().replace('%', '')) / 100, line[1:]))
    for hour, percentage in zip(header, percentages):
        if hour not in result:
            result[hour] = []
        result[hour].append(percentage)
print(result)

{'100': [0.79, 0.67, 0.59, 0.52, 0.47, 0.42, 0.38, 0.35, 0.31, 0.28, 0.26, 0.23, 0.21, 0.19, 0.17, 0.16, 0.14, 0.13, 0.11, 0.1], '300': [0.69, 0.58, 0.5, 0.44, 0.39, 0.35, 0.31, 0.28, 0.25, 0.23, 0.2, 0.18, 0.16, 0.14, 0.13, 0.11, 0.1, 0.09, 0.08, 0.07], '500\n': [0.63, 0.52, 0.45, 0.39, 0.34, 0.3, 0.27, 0.24, 0.21, 0.19, 0.17, 0.15, 0.13, 0.12, 0.1, 0.09, 0.08, 0.07, 0.06, 0.05]}


In [30]:
data = open_csv('../static/remaining_value_tractor.csv')
result = {}
hps = data[0][1:]
hours = data[1][1:]

for line in data[2:]:
    percentages = list(map(lambda v: float(v.strip().replace('%', '')) / 100, line[1:]))
    for hour, hp, percentage in zip(hours, hps, percentages):
        if hour not in result:
            result[hour] = {}
        if hp not in result[hour]:
            result[hour][hp] = []
        result[hour][hp].append(percentage)
print(result)

{'200': {'30': [0.65, 0.59, 0.54, 0.51, 0.48, 0.45, 0.42, 0.4, 0.38, 0.36, 0.35, 0.33, 0.32, 0.3, 0.29, 0.28, 0.26, 0.25, 0.24, 0.23], '40': [0.65, 0.59, 0.54, 0.51, 0.48, 0.45, 0.42, 0.4, 0.38, 0.36, 0.35, 0.33, 0.32, 0.3, 0.29, 0.28, 0.26, 0.25, 0.24, 0.23], '50': [0.65, 0.59, 0.54, 0.51, 0.48, 0.45, 0.42, 0.4, 0.38, 0.36, 0.35, 0.33, 0.32, 0.3, 0.29, 0.28, 0.26, 0.25, 0.24, 0.23], '60': [0.65, 0.59, 0.54, 0.51, 0.48, 0.45, 0.42, 0.4, 0.38, 0.36, 0.35, 0.33, 0.32, 0.3, 0.29, 0.28, 0.26, 0.25, 0.24, 0.23], '70': [0.65, 0.59, 0.54, 0.51, 0.48, 0.45, 0.42, 0.4, 0.38, 0.36, 0.35, 0.33, 0.32, 0.3, 0.29, 0.28, 0.26, 0.25, 0.24, 0.23], '80': [0.69, 0.62, 0.57, 0.53, 0.5, 0.47, 0.44, 0.42, 0.4, 0.38, 0.36, 0.34, 0.33, 0.31, 0.3, 0.28, 0.27, 0.26, 0.25, 0.24], '90': [0.69, 0.62, 0.57, 0.53, 0.5, 0.47, 0.44, 0.42, 0.4, 0.38, 0.36, 0.34, 0.33, 0.31, 0.3, 0.28, 0.27, 0.26, 0.25, 0.24], '100': [0.69, 0.62, 0.57, 0.53, 0.5, 0.47, 0.44, 0.42, 0.4, 0.38, 0.36, 0.34, 0.33, 0.31, 0.3, 0.28, 0.27, 0.26

In [ ]:
data = open_csv('../static/repair_costs.csv')
result = {}
hours = list(map(lambda v: v.strip(), data[0][1:]))
for line in data[1:]:
    category = normalize_text(line[0]).replace(' ', '_')
    percentages = list(map(lambda v: round(float(v.strip().replace('%', '')) / 100, 4), filter(lambda v: v.strip() != '', line[1:])))
    if category not in result:
        result[category] = {}
    for value, hour in zip(percentages, hours[0:len(percentages)]):
        result[category][hour] = value


dict_keys(['25', '50', '100', '200', '300', '400', '500', '600', '700', '800', '900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200', '2300', '2400', '2500', '2600', '2700', '2800', '2900', '3000', '4000', '5000', '6000', '7000', '8000', '9000', '10000'])


In [27]:
list(result.keys())

['traktors_4x2',
 'traktors_4x4',
 'plough',
 'smagie_diski',
 'tandema_disks',
 'cizelarkls_700',
 'lauku_kultivators',
 'ecesas',
 'rullveida_veltnis_mulcetajs',
 'rotejosas_ecesas',
 'rindstarpu_kultivators',
 'heders_uzkarinams',
 'kartupelu_novaksanas_kombains',
 'plaujmasinakondicionieris',
 'plaujmasinakondicionieris_rotacijas',
 'siena_grabeklis',
 'kipu_prese',
 'kipu_prese_liela',
 'lopbaribas_novaksanas_kombains_velkamais',
 'lopbaribas_kombains_sp',
 'kombains_sp',
 'pasgajejkombains_sp',
 'kokvilnas_novaksanas_kombains_sp',
 'plaujmasina_piku',
 'plaujmasinas_rotacijas',
 'lielo_ritulu_prese',
 'cukurbiesu_kombains',
 'rindu_staditajs',
 'graudu_sejmasina',
 'meslojuma_izkliedetajs',
 'stangu_tipa_smidzinatajs',
 'smidzinatajs_ar_pneimatiskais',
 'pupinu_novilcejsvilcejs',
 'stublaju_smalcinatajs',
 'lopbaribas_putejs',
 'piekabe',
 'lopbaribas_piekabe']

In [26]:
' | '.join(map(lambda v: f"'{v}'", list(result.keys())))

"'traktors_4x2' | 'traktors_4x4' | 'plough' | 'smagie_diski' | 'tandema_disks' | 'cizelarkls_700' | 'lauku_kultivators' | 'ecesas' | 'rullveida_veltnis_mulcetajs' | 'rotejosas_ecesas' | 'rindstarpu_kultivators' | 'heders_uzkarinams' | 'kartupelu_novaksanas_kombains' | 'plaujmasinakondicionieris' | 'plaujmasinakondicionieris_rotacijas' | 'siena_grabeklis' | 'kipu_prese' | 'kipu_prese_liela' | 'lopbaribas_novaksanas_kombains_velkamais' | 'lopbaribas_kombains_sp' | 'kombains_sp' | 'pasgajejkombains_sp' | 'kokvilnas_novaksanas_kombains_sp' | 'plaujmasina_piku' | 'plaujmasinas_rotacijas' | 'lielo_ritulu_prese' | 'cukurbiesu_kombains' | 'rindu_staditajs' | 'graudu_sejmasina' | 'meslojuma_izkliedetajs' | 'stangu_tipa_smidzinatajs' | 'smidzinatajs_ar_pneimatiskais' | 'pupinu_novilcejsvilcejs' | 'stublaju_smalcinatajs' | 'lopbaribas_putejs' | 'piekabe' | 'lopbaribas_piekabe'"

In [8]:
import re, json, os
replacements = {
    'ā': 'a',
    'ē': 'e',
    'č': 'c',
    'ī': 'i',
    'ļ': 'l',
    'š': 's',
    'ū': 'u',
    'ģ': 'g',
    'ķ': 'k',
    'ž': 'z',
    'ņ': 'n'
}

def open_json(file):
    data = {}
    if os.path.exists(file):
        with open(file, 'r', encoding="UTF-8") as fp:
            data = json.load(fp)
            fp.close()
    return data
def save_json(file, data):
    with open(file, 'w', encoding="UTF-8") as fp:
        json.dump(data, fp, indent=4)
        fp.close()

def open_csv(file, sep=';'):
    fs = open(file, 'r', encoding='utf-8')
    data = list(map(lambda row: row.split(sep),fs.readlines()))
    fs.close()
    return data

def normalize_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    for r in replacements:
        if r in text:
            text = text.replace(r, replacements[r])
    return text

data = open_json("../data/mascus/mascus_short.json")

analysis = {}
for listing in data:
    category = listing['mascusCategory']
    if category not in analysis:
        analysis[category] = {
            "by_years": {},
            "by_mark_model": {}
        }
    year = listing['manufactureYear']
    if year not in analysis[category]["by_years"]:
        analysis[category]["by_years"][year] = []
    analysis[category]["by_years"][year].append(listing)

    mark_model = normalize_text(f"{listing['mark']} {listing['model']}")
    if mark_model not in analysis[category]["by_mark_model"]:
        analysis[category]["by_mark_model"][mark_model] = []
    analysis[category]["by_mark_model"][mark_model].append(listing)
save_json("../data/mascus/analysis.json", analysis)

In [18]:
import requests
from bs4 import BeautifulSoup

def fetch_mark_links():
    url = "https://www.tractordata.com/farm-tractors/index.html"
    res = requests.get(url)

    soup = BeautifulSoup(res.content, features="html.parser")
    table =  soup.find("table", attrs={'class': 'tdMenu1'})

    marks = {}
    for row in table.find_all("tr"):
        cells = row.find_all("td")
        link = cells[0].find("a")
        mark = link.text
        marks[mark] = link.attrs["href"]
    return marks
global marks
marks = fetch_mark_links()

In [42]:
import re
def fetch_mark(mark):
    global marks
    if mark not in marks:
        raise Exception(f"{mark} does not exist")
    url = marks[mark]
    res = requests.get(url)
    soup = BeautifulSoup(res.content, features="html.parser")
    table = soup.find("table", attrs={'class': 'tdMenu1'})
    models = {}
    for row in table.find_all("tr")[1:]:
        cells = row.find_all("td")
        link = cells[0].find("a")
        if link is None: 
            continue
        model = link.text
        start_year = re.match("^[0-9]{4}", cells[2].text.strip())
        end_year = re.match("[0-9]{4}$", cells[2].text.strip())
        
        models[model] = {
            "link": link.attrs["href"],
        }

        if start_year is not None:
            models[model]["start_year"] = start_year[0]
        
        if end_year is not None:
            models[model]["end_year"] = end_year[0]
    return models
models = fetch_mark("John Deere")

In [52]:
def fetch_model(model, url=None):
    if url is None:
        url = model["link"]
    res = requests.get(url)
    soup = BeautifulSoup(res.content, features="html.parser")
    return soup

data = fetch_model(models["6120M"])
        


    

In [59]:
def fetch_tests(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, features="html.parser")

    return soup.find_all("div", attrs={"class": "tdArticleItem"})

a = fetch_tests("https://www.tractordata.com/farm-tractors/008/9/8/8986-john-deere-6120m-tests.html")

In [65]:
data = open_json("../data/mascus/mascus.json")
real = {}
for item in data:
    if item["productId"] not in real:
        real[item["productId"]] = item
save_json("../data/mascus/mascus.json", real)

In [58]:

year_select = data.find("select", attrs={"name": "SelectYear"})
specifications = []
if year_select is not None:
    version_options = {
       option.text : option.attrs["value"] for option in year_select.find_all("option")
    }
navs = {
    nav.text : nav.attrs["href"] for nav in data.find("div", attrs={"class": "tractornav"}).find_all("a")
}

print(navs)

{'Overview': '#', 'Engine': 'https://www.tractordata.com/farm-tractors/008/9/8/8986-john-deere-6120m-engine.html', 'Transmission': 'https://www.tractordata.com/farm-tractors/008/9/8/8986-john-deere-6120m-transmission.html', 'Dimensions': 'https://www.tractordata.com/farm-tractors/008/9/8/8986-john-deere-6120m-dimensions.html', 'Photos': 'https://www.tractordata.com/farm-tractors/008/9/8/8986-john-deere-6120m-photos.html', 'Tests': 'https://www.tractordata.com/farm-tractors/008/9/8/8986-john-deere-6120m-tests.html', 'Attachments': 'https://www.tractordata.com/farm-tractors/008/9/8/8986-john-deere-6120m-attachments.html'}


In [34]:
import numpy as np
import pandas as pd
data = open_json("../data/mascus/mascus_short.json")
hours_results = {}
for listing in data:
    category = listing["mascusCategory"]
    year = listing["manufactureYear"]
    price = listing["price"]
    hours = listing["motorHours"]
    if year is None:
        continue
    if category not in hours_results:
        hours_results[category] = {}
    if year not in hours_results[category]:
        hours_results[category][year] = []
    if hours is not None:
        hours_results[category][year].append(hours)

averages = {}
for category, years in hours_results.items():
    if category not in averages:
        averages[category] = {}
    for year, values in years.items():
        if len(values) > 0:
            averages[category][year] = np.average(np.array(values)).item()

for category, year_data in averages.items():
    if len(year_data.keys()) <= 0:
        continue
    df = pd.DataFrame(list(year_data.items()), columns=["Year", "Value"]).sort_values("Year")
    min_year, max_year = int(df["Year"].min()), int(df["Year"].max())
    df = df.set_index("Year").reindex(range(min_year, max_year + 1))
    
    df["Value"] = df["Value"].interpolate(method="linear")
    
    df["Value"] = df["Value"].round(-2)
    
    averages[category] = df["Value"].dropna().to_dict()

print(averages)

{'piekabinamie_pasgajeji_smidzinataji(miglotaji)': {2010: 1400.0, 2011: 23800.0, 2012: 16000.0, 2013: 8200.0, 2014: 4700.0, 2015: 1200.0, 2016: 2200.0, 2017: 3300.0, 2018: 1300.0, 2019: 600.0, 2020: 500.0, 2021: 400.0, 2022: 200.0}, 'urbji_agri': {1995: 1700.0, 1996: 1800.0, 1997: 1800.0, 1998: 1800.0, 1999: 1800.0, 2000: 1800.0, 2001: 1900.0, 2002: 1900.0, 2003: 1900.0, 2004: 1900.0, 2005: 2000.0, 2006: 2000.0, 2007: 2000.0, 2008: 2000.0, 2009: 2900.0, 2010: 3800.0, 2011: 4700.0, 2012: 5600.0, 2013: 600.0, 2014: 2800.0, 2015: 2000.0, 2016: 1800.0, 2017: 1300.0, 2018: 400.0, 2019: 100.0, 2020: 700.0, 2021: 400.0, 2022: 100.0, 2023: 100.0, 2024: 0.0}, 'mineralmeslu_izkliedetaji': {1995: 4300.0, 1996: 4100.0, 1997: 3900.0, 1998: 3700.0, 1999: 3400.0, 2000: 3200.0, 2001: 3000.0, 2002: 2800.0, 2003: 2600.0, 2004: 2400.0, 2005: 2100.0, 2006: 1900.0, 2007: 1700.0, 2008: 1500.0, 2009: 1300.0, 2010: 1000.0, 2011: 800.0, 2012: 600.0, 2013: 400.0, 2014: 2800.0, 2015: 2700.0, 2016: 400.0, 2017: 1

In [59]:
specifications = open_json("../data/specifications.json")
def flatten(specifications):
    result = []
    for category in specifications:
        for manufacturer in category["manufacturers"]:
            if "models" not in manufacturer:
                continue
            for model in manufacturer["models"]:
                model["category"] = category["industryEquipment"]
                model["manufacturer"] = manufacturer["manufacturerName"]
                result.append(model)
    return result
result = flatten(specifications)
print(len(result))
categories = []
for item in result:
    if item["category"] not in categories:
        categories.append(item["category"])
print(categories)

3729
['2WD Tractor', '4WD Tractor', 'Air Drill', 'Baler', 'Combine', 'Cultivator', 'Disc', 'Harrow', 'MFWD Tractor', 'Sprayer', 'Swather', 'Utility Tractor']


In [60]:

def extract_subparam_names(data, subparam_names=None):
    if subparam_names is None:
        subparam_names = []
    
    if isinstance(data, dict):
        if "subparam" in data and isinstance(data["subparam"], str):
            subparam_names.append(data["subparam"])
        for value in data.values():
            extract_subparam_names(value, subparam_names)
    elif isinstance(data, list):
        for item in data:
            extract_subparam_names(item, subparam_names)
    
    return subparam_names
all_params = []

def flatten_specifications(data, specifications=None):
    if specifications is None:
        specifications = []
    
    if isinstance(data, dict):
        if "subparam" in data and isinstance(data["subparam"], str):
            spec_entry = {"name": data["subparam"]}
            for key, value in data.items():
                if key != "subparam":
                    spec_entry[key] = value
            specifications.append(spec_entry)
        for value in data.values():
            flatten_specifications(value, specifications)
    elif isinstance(data, list):
        for item in data:
            flatten_specifications(item, specifications)
    
    return specifications
for item in result:
    if "specifications" not in item:
        continue
    for param in extract_subparam_names(item["specifications"]):
        
        if param not in all_params:
            all_params.append(param)

print(all_params)

['Number Of Cylinders', 'Engine Make', 'Pto Power', 'Power Measured @', 'Displacement', 'Aspiration', 'Operating Weight', 'Fuel Capacity', '3 Point Hitch Lift Capacity @ 24 Inches', 'Transmission Type', 'Number Of Forward Gears', 'Number Of Reverse Gears', 'Pump Flow Capacity', 'Number Of Remote Control Valves', 'Gross Power', 'Turning Radius', 'Engine Model', 'Operating Voltage', 'Alternator Supplied Amperage', 'Relief Valve Setting', 'Tire Size Front - 2wd / 4wd', 'Rear Tires Size 2wd/4wd', 'Max Speed', 'Max Torque', 'Torque Measured @', 'Engine Oil Capacity', 'Cooling System Fluid Capacity', 'Hydraulic System Fluid Capacity', 'Fuel Usage @ 75% Load, Full Rpm', 'Net Power', 'Engine Type', 'Rated Speed', 'Bore', 'Stroke', '3 Point Hitch Type', 'Lift Configuration', 'Brakes', 'Steering', 'Clutch, Wet/Dry', 'Implement Pump Flow Ps', 'System Pressure', 'Air Cleaner', 'Alternator', 'Pto Speed', 'Power Take-Off(Pto) Type', 'Shuttle (Forward-Reverse)', 'Brake Type', 'Clutch Disk Diameter We

In [61]:
for item in result:
    if "specifications" not in item:
        continue
    specifications = flatten_specifications(item["specifications"])

In [53]:
catalog = open_json('../data/catalog_data.json')
categories = []
for item in catalog:
    if item["EquipmentTypeCode"] not in categories:
        categories.append(item["EquipmentTypeCode"])

In [55]:
print(categories)

['traktors_4x4', 'sejmasina', 'plaujmasina', 'veltnis', 'mulceris', 'rindstarpu_kultivators', 'rugaines_kultivators', 'organisko_meslu_izkliedetajs', 'mineralmeslu_un_kalka_izkliedetajs', 'sikseklu_sejmasina', 'diski', 'valotajsarditajs', 'smidzinatajs', 'ecesas', 'dzilirdinatajs', 'miglotajs', 'arkls', 'ritulu_prese', 'skidrmeslu_cisterna', 'precizas_izsejas_sejmasina', 'freze', 'kipu_prese', 'kultivators', 'skabbaribas_blietetajveltnis', 'salmu_smalcinatajsizkliedetajs', 'ogu_novaksans_kombains', 'graudaugu_kombains', 'skidrmeslu_izkliedes_caurulvadu_sistema', 'skidrmeslu_izkliedetaja_aprikojums', 'ietinejs', 'laistisanas_iekarta', 'lakstu_plavejs', 'darzenu_novaksanas_kombains', 'maurina_plaujmasina', 'auglu_koku_un_ogulaju_stadama_masina', 'kartupelu_stadama_masina', 'akmenu_vacejs', 'pleves_ieklajejs', 'trimmeris', 'stadu_konteineru_pildisanas_un_stadisanas_iekarta', 'kartupelu_novaksanas_kombains', 'kartupelu_vagotajs', 'prese_ar_ietineju', 'komposta_apversejs', 'darzenu_stadama_